<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Try id #259

In [1]:
import sys
sys.path.append('..')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [2]:
import taiko as tk
from taiko.play import *
from taiko.performance import *
from taiko.model import *
from taiko.database import *
from taiko.tools.config import *
from taiko.tools.score import *
from taiko.visualize import *

from collections import deque
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import posixpath
from scipy.stats import mode
from skimage.io import imshow, imsave, imread
import multiprocessing
from tqdm import tqdm

sns.set(font_scale=1.5)

Using TensorFlow backend.


In [9]:
record_id = 211
lgbm = LGBM()
record_df = load_record_df()
record_row = record_df.loc[record_id]
play = get_play(record_row)
pf = get_performance(play, id_=record_id)

train_df = pf.copy()
x = train_df.drop('timestamp', axis=1)
ts = train_df['timestamp']
hit_type_x = lgbm.predict(x)
train_df['hit_type'] = hit_type_x

pred_df = pd.DataFrame(data={
    'timestamp': ts,
    'hit_type': hit_type_x
})

pred_ptr = 0
pred_pool = deque([0] * 5)
song_score = 0
hit_count = 0

pred_hit_types = []
true_hit_types = []
for event_time, hit_type in play.events:
    while pred_df.loc[pred_ptr].timestamp <= event_time + 0.1:
        pred_hit_type = int(pred_df.loc[pred_ptr].hit_type)
        pred_hit_type = transform_hit_type(pred_hit_type)
        pred_pool.popleft()
        pred_pool.append(pred_hit_type)
        pred_ptr += 1

    pool_hit_type = mode(pred_pool)[0]
    print(pred_pool)
    print(pool_hit_type, hit_type)
    multiplier = get_gained_score_multiplier(pool_hit_type, hit_type)
    if multiplier > 0:
        hit_count += 1
        song_score += SCORE_UNIT_DICT[1] * multiplier

print(song_score, hit_count)

deque([2, 2, 2, 2, 2])
[2] 1
deque([2, 2, 2, 2, 2])
[2] 0
deque([2, 0, 0, 0, 0])
[0] 0
deque([0, 0, 0, 0, 2])
[0] 0
deque([2, 2, 2, 2, 2])
[2] 1
deque([2, 2, 2, 2, 2])
[2] 0
deque([0, 0, 0, 0, 0])
[0] 0
deque([0, 0, 2, 2, 2])
[2] 0
deque([2, 2, 2, 2, 2])
[2] 1
deque([2, 2, 2, 4, 4])
[2] 0
deque([4, 0, 2, 1, 1])
[1] 1
deque([1, 1, 1, 2, 2])
[1] 0
deque([3, 3, 3, 2, 2])
[3] 1
deque([2, 2, 2, 2, 2])
[2] 0
deque([4, 2, 0, 0, 0])
[0] 0
deque([3, 0, 3, 3, 3])
[3] 0
deque([0, 1, 1, 1, 1])
[1] 1
deque([1, 1, 1, 1, 1])
[1] 0
deque([2, 0, 0, 0, 0])
[0] 0
deque([0, 0, 2, 0, 1])
[0] 0
deque([2, 2, 2, 1, 1])
[2] 1
deque([2, 1, 2, 1, 0])
[1] 0
deque([0, 0, 0, 0, 0])
[0] 0
deque([0, 0, 0, 1, 1])
[0] 0
deque([2, 1, 3, 5, 5])
[5] 5
deque([5, 3, 5, 5, 5])
[5] 5
deque([5, 5, 5, 5, 5])
[5] 5
deque([5, 5, 5, 5, 5])
[5] 5
deque([5, 3, 5, 5, 5])
[5] 5
deque([5, 5, 5, 5, 3])
[5] 5
deque([5, 5, 3, 5, 5])
[5] 0
deque([5, 5, 5, 3, 3])
[5] 0
deque([0, 1, 1, 1, 1])
[1] 1
deque([1, 1, 1, 1, 2])
[1] 0
deque([0, 0, 0

deque([1, 1, 1, 1, 1])
[1] 1
deque([1, 2, 2, 2, 1])
[2] 0
deque([0, 0, 0, 0, 0])
[0] 0
deque([0, 1, 1, 1, 1])
[1] 0
deque([1, 1, 1, 1, 1])
[1] 1
deque([1, 1, 2, 1, 2])
[1] 0
deque([0, 0, 1, 1, 1])
[1] 1
deque([1, 1, 2, 2, 2])
[2] 0
deque([0, 0, 0, 0, 0])
[0] 0
deque([0, 0, 0, 0, 0])
[0] 0
deque([0, 0, 0, 0, 0])
[0] 0
deque([0, 0, 0, 2, 2])
[0] 0
deque([2, 2, 1, 1, 1])
[1] 1
deque([1, 1, 1, 2, 1])
[1] 0
deque([3, 0, 1, 1, 1])
[1] 1
deque([1, 2, 1, 2, 2])
[2] 0
deque([1, 1, 2, 1, 2])
[1] 1
deque([1, 1, 2, 1, 0])
[1] 0
deque([0, 0, 0, 0, 0])
[0] 0
deque([0, 0, 0, 0, 0])
[0] 0
deque([0, 1, 1, 1, 1])
[1] 1
deque([1, 1, 1, 2, 1])
[1] 0
deque([0, 0, 1, 1, 1])
[1] 1
deque([1, 1, 1, 1, 1])
[1] 0
deque([1, 0, 0, 0, 0])
[0] 0
deque([0, 0, 0, 0, 0])
[0] 0
deque([0, 0, 0, 0, 0])
[0] 0
deque([0, 0, 0, 1, 1])
[0] 0
deque([1, 1, 1, 1, 1])
[1] 1
deque([1, 1, 1, 1, 1])
[1] 0
deque([3, 0, 1, 1, 1])
[1] 1
deque([1, 1, 1, 1, 1])
[1] 0
deque([0, 0, 1, 1, 1])
[1] 1
deque([1, 1, 1, 1, 0])
[1] 0
deque([0, 0, 0

In [4]:
record_df = load_record_df(song_id=1)
# record_df = record_df[(record_df['song_id'] >= 1) & (record_df['song_id'] <= 4)]
record_df

,drummer_name,song_id,left_sensor_datetime,right_sensor_datetime,capture_datetime
1,aaaaa,1,L_2018-09-28_111304.csv,R_2018-09-28_111305.csv,capture_2018_09_28_11_13_05
5,aaaaa,1,L_2018-09-28_112912.csv,R_2018-09-28_112913.csv,capture_2018_09_28_11_29_13
9,aaaaa,1,L_2018-09-28_114340.csv,R_2018-09-28_114340.csv,capture_2018_09_28_11_43_41
13,aaaaa,1,L_2018-09-28_115822.csv,R_2018-09-28_115823.csv,capture_2018_09_28_11_58_23
25,carolyn,1,L_2018-09-27_154913.csv,R_2018-09-27_154916.csv,capture_2018_09_27_15_49_16
29,carolyn,1,L_2018-09-27_160509.csv,R_2018-09-27_160510.csv,capture_2018_09_27_16_05_10
33,carolyn,1,L_2018-09-27_161825.csv,R_2018-09-27_161826.csv,capture_2018_09_27_16_18_26
37,carolyn,1,L_2018-09-27_163110.csv,R_2018-09-27_163111.csv,capture_2018_09_27_16_31_11
42,celiven,1,L_2018-09-22_141822.csv,R_2018-09-22_141821.csv,capture_2018_09_22_14_18_21
46,celiven,1,L_2018-09-22_142958.csv,R_2018-09-22_142957.csv,capture_2018_09_22_14_29_57


In [5]:
lgbm = LGBM()
for id_ in reversed(record_df.index):
    try:
        print(id_, predict_score(id_, lgbm))
    except Exception:
        print(id_, 'error!!!')
# with multiprocessing.Pool(1) as p:
#     for id_, _ in tqdm(enumerate(p.imap(predict_score, record_df.index)), total=len(record_df.index)):
#         pass

KeyboardInterrupt: 